In [1]:
# %%
#To Run the code save the code ,"Inpute.xlsx" ,MasterDictionary,StopWords in same folder#
#The code will genarate URL_ID.txt for each URL id and save it in same directory 
#Output data structure.xlsx file will be genarated in same directory


#My approach is to get data and sub set it in small set to begin with after gathering appropriate data we will build 
#function to calculate each feature we want to extract
# *Note* I did initial testing on small set of given data to get my logic correct 


#First thing we have to extract the data from given file "Inpute.xlsx" as input file to extract the URLs and URL_IDs
#save these in respective list

#Import required libraries  
import requests
import nltk
import numpy as np 
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import string
# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download("vader_lexicon")
import os  
import re            
sia = SentimentIntensityAnalyzer()
from bs4 import BeautifulSoup         #Using BeautifulSoup to scrap the text
import pandas as pd

df = pd.read_excel('Input.xlsx')

url_list = df['URL'].tolist()
url_id = df['URL_ID'].tolist()

#Function to scrap the URL
article_title=''
article_text=''
extracted_text=[]
def extract_text_from_url(url):
        res = requests.get(url)
        html_page = res.content

        soup = BeautifulSoup(html_page, 'html.parser')
        article_title = soup.title
        paragraphs = soup.find_all('p')
        article_text = ' '.join(p.text for p in paragraphs)
        text = str(article_title)+str(article_text)
        return text 
        # Inside the loop
      

#Extract the text from url
for url in url_list:
    extracted_text = extract_text_from_url(url)
     



#Save data from input file
def save_text_to_file(text, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(str(text))

for i, url in enumerate(url_list):
    id=url_list[i]
    extracted_text = extract_text_from_url(url)
    #print(extracted_text)
    save_text_to_file(extracted_text, f'{url_id[i]}.txt')
    print(url_id[i]+' Is Genarated in same folder !')

#Import positive and negative words from given dictionaries 
with open('MasterDictionary/negative-words.txt') as f :
    neg_word = f.read().splitlines()

with open('MasterDictionary/positive-words.txt') as f :
    pos_word = f.read().splitlines()

#Setup all inicially null varieables
res = {}
positive_score = {}
negative_score = {}
polarity_score = {}
subjectivity_score ={}
avgsen ={}
complex_word_count = {}
Fog_Index={}
Avg_num_word_per_sen={}
Per_comp_word={}
clean_word_count ={}
SWP={}
Total_pronoun_counts={}
Average_Word_Length={}
pos_score = 0
neg_score = 0
comp_count = 0
path ="StopWords"       #Import Stopwolds file names
stop_list = os.listdir(path)
stop_list

#Polarity function
def pol_score(pos_score, neg_score):
  # Calculate the polarity score
  pol = (pos_score - neg_score) / ((pos_score + neg_score) + 0.000001)
  # Return the polarity score
  return pol

#Subjectivity Function
def subjectivity(pos_score,neg_score):
    sub =  (pos_score + neg_score) / (len(clean_text) + 0.000001)
    return sub

# Define the average sentence length function
def averageSentence(text):
    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    # Get the number of sentences
    num_sentences = len(sentences)
    # Initialize a variable for the number of words
    num_words = 0
    # Loop through the sentences and count the words
    for sentence in sentences:
        # Split the sentence into words
        words = sentence.split()
        # Get the number of words in the sentence
        num_words += len(words)
    # Calculate the average sentence length
    average = num_words / num_sentences
    # Return the average sentence length
    return average

#Complex Word Count
def count_complex_words(text):
    d = dict();
    # Initialize the count to zero
    count = 0
    # Split the text into words by whitespace
    words = text.split()
    # Loop through each word
    for word in words:
            # Remove any punctuation from the word
            word = word.strip(". , ? ! : ; ' \" ( ) [ ] { }")
            # Convert the word to lowercase
            word = word.lower()
            # Count the number of vowels in the word
            vowels = "aeiouy"
            vowel_count = 0
            for letter in word:
                if letter in vowels:
                    vowel_count += 1
            # Subtract any silent vowels, such as the final 'e'
            if word.endswith("e"):
                vowel_count -= 1
            # Add one if the word ends with a vowel followed by a consonant
            if len(word) > 2 and word[-2] in vowels and word[-1] not in vowels:
                vowel_count += 1
            # Divide the vowel count by 2 to get the number of syllables
            syllable_count = vowel_count // 2
            # If the word has more than 3 syllables, it is a complex word
            if syllable_count > 1:
                    count += 1
                
        # Return the count of complex words
    return count

#Percentage of complex word function
def Percentage_complex_word(text):
    #print(text)
    sentences = nltk.sent_tokenize(text)
        # Get the number of sentences
    num_sentences = len(sentences)
        # Initialize a variable for the number of words
    num_words = 0
        # Loop through the sentences and count the words
    for sentence in sentences:
        # Split the sentence into words
        words = sentence.split()
        # Get the number of words in the sentence
        num_words += len(words)
    per_comp_word = count_complex_words(text) / num_words
    per_comp_word = per_comp_word*100
    fog_index = 0.4* (avg_sen + per_comp_word)
    avg_num_word_per_sen = num_words/num_sentences
    return [per_comp_word,fog_index,avg_num_word_per_sen] 

#Clean text for word counting
def clean(text):
    # Download NLTK stopwords if not already downloaded
    nltk.download('stopwords')
    
    # Get the list of English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Tokenize the input text into words
    words = text.split()
    
    # Remove punctuation and filter out stopwords
    cleaned_words = [word.strip(string.punctuation) for word in words if word.lower() not in stop_words]
    cleaned_words_count =len(cleaned_words)
    return cleaned_words_count

#Count Yllable function
def count_syllables(word):
  # Convert the word to lowercase
  word = word.lower()
  # Initialize the syllable count to zero
  syllables = 0
  # Define the vowels
  vowels = "aeiouy"
  # Loop through each letter in the word
  for i in range(len(word)):
    # If the letter is a vowel
    if word[i] in vowels:
      # Increment the syllable count
      syllables += 1
      # If the letter is the last one and it is 's'
      if i == len(word) - 1 and word[i-1] == 'e' and word[i] == 's':
        # Decrement the syllable count
        syllables -= 1
      # If the letter is the last one and it is 'd'
      if i == len(word) - 1 and word[i-1] == 'e' and word[i] == 'd':
        # Decrement the syllable count
        syllables -= 1
      # If the letter is not the first one and the previous letter is also a vowel
      if i>0 and word[i-1] in vowels:
        # Decrement the syllable count
        syllables -= 1
  # If the syllable count is zero, set it to one
  if syllables == 0:
    syllables = 1
  # Return the syllable count per word 
    # Return the syllable count
  return syllables

#Syllable Count Per Word
# Define a function to count the syllables in each word of a text
def count_syllables_per_word(text):
  # Split the text into words by whitespace
  words = text.lower().split()
  # Initialize an empty dictionary to store the word-syllable pairs
  word_syllables = {}
  # Loop through each word in the text
  for word in words:
    # Remove any punctuation from the word
    word = word.strip(",.!?;:")
    # Count the syllables in the word
    syllables = count_syllables(word)
    # Add the word-syllable pair to the dictionary
    word_syllables[word] = syllables
    syll_per_word = sum(word_syllables.values())/num_words
  return syll_per_word

# Define a function to count the personal pronouns in a text
def count_personal_pronouns(text):
  # Convert the text to lowercase
  # Define the personal pronouns
  pronouns = ["I", "we", "my", "ours", "us"]
  # Initialize an empty dictionary to store the pronoun-count pairs
  pronoun_counts = {}
  # Loop through each pronoun
  for pronoun in pronouns:
    # Use regex to find the occurrences of the pronoun in the text
    # Use word boundaries to avoid matching partial words
    # Use negative lookahead to exclude the country name US
    matches = re.findall(r"\b" + pronoun + r"\b(?!-?US\b)" , text)
    # Count the number of matches
    count = len(matches)
    # Add the pronoun-count pair to the dictionary
    pronoun_counts[pronoun] = count        #personal pronoun count is saved in dictionary name pronoun_count
  pp = sum(dict.values(pronoun_counts))
  # Return the dictionary
  return pp


# Define a function to calculate the average word length in a text
def average_word_length(text):
  # Split the text into words by whitespace
  words = text.split()
  # Get the number of words in the text
  num_words = len(words)
  # Initialize the total number of characters to zero
  total_chars = 0
  # Loop through each word in the text
  for word in words:
    # Remove any punctuation from the word
    word = word.strip(",.!?;:")
    # Add the length of the word to the total number of characters
    total_chars += len(word)
  # Calculate the average word length by dividing the total number of characters by the number of words
  average = total_chars / num_words
  # Return the average word length
  return average

#Loop to read Extracted data then send them to require functions to create Output Data Structure.xlsx in given directory
for i,url in enumerate(url_list):       
    file = open(f'{url_id[i]}.txt' , encoding="utf8")
    text = file.read()

    for j ,sl in enumerate(stop_list) :
            with open(f'{stop_list[j]}') as f:
                stopwords = f.read().splitlines() # Contains "and" and "or" on different lines
            tokens = text.split() # Split into list of words
            for word in tokens: 
                if word.lower() in stopwords: # If word in stopwords remove it
                    tokens.remove(word)
            clean_text = " ".join(word for word in tokens) # Join words into a string


     #   with open("og_text{i}", 'w', encoding='utf-8') as file:
     #           file.write(str(text))
     #    with open("clean_text", 'w', encoding='utf-8') as file:
     #           file.write(str(clean_text))

        #pos_word
            for word in tokens :
                if word.lower() in pos_word : pos_score += 1        #Check for positive word in dictionary
               
        #print(word)
            for word in tokens :
                if word.lower() in neg_word :                       #Check for negative word in dictionary
                    neg_score -= 1 *-1                              #Multiplying by -1

    positive_score[i] =pos_score
    negative_score[i]=neg_score                                 
    df['Positive score'] = df.index.map(positive_score)             #Storing positive score
    df['Nagative score'] = df.index.map(negative_score)             #Storing negative score
    #print(positive_score[i])
    #print(pos_score)
    #print(neg_score)

    pol = pol_score(pos_score, neg_score) # Polarity score storing
    polarity_score[i] = pol
    df['Polarity score'] = df.index.map(polarity_score) 
    
    #Subjectivity score storing
    sub = subjectivity(pos_score, neg_score) 
    subjectivity_score[i]=sub
    df['Subjectivity score'] = df.index.map(subjectivity_score)

    #avg_sen_len = avg_sen_len()
    avg_sen = averageSentence(text)
    avgsen[i]=avg_sen
    df['AVG SENTENCE LENGTH'] = df.index.map(avgsen)

    pc = Percentage_complex_word(text)
    Per_comp_word[i]=pc[0]
    Fog_Index[i] = pc[1]
    Avg_num_word_per_sen[i] = pc[2]
    df['PERCENTAGE OF COMPLEX WORDS'] = df.index.map(Per_comp_word)
    df['Fog Index'] = df.index.map(Fog_Index)
    df['AVG NUMBER OF WORDS PER SENTENCE'] = df.index.map(Avg_num_word_per_sen)

    cx = count_complex_words(text)
    complex_word_count[i]=cx
    df['COMPLEX WORD COUNT'] = df.index.map(complex_word_count)

    # Download NLTK stopwords if not already downloaded
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    
    # Get the list of English stopwords
    stopwords = set(stopwords.words('english'))
    
    # Tokenize the input text into words
    words = text.split()
    
    # Remove punctuation and filter out stopwords
    cleaned_words = [word.strip(string.punctuation) for word in words if word.lower() not in stopwords]
    c=len(cleaned_words)
    clean_word_count[i]=c
    df['WORD COUNT'] = df.index.map(clean_word_count)



    sentences = nltk.sent_tokenize(text)
        # Get the number of sentences
    num_sentences = len(sentences)
        # Initialize a variable for the number of words
    num_words = 0
        # Loop through the sentences and count the words
    for sentence in sentences:
        # Split the sentence into words
        words = sentence.split()
        # Get the number of words in the sentence
        num_words += len(words)


    spw = count_syllables_per_word(text)
    SWP[i]=spw
    df['SYLLABLE PER WORD'] = df.index.map(SWP)

    total_pro_counts = count_personal_pronouns(text)
    Total_pronoun_counts[i] = total_pro_counts
    df['PERSONAL PRONOUNS'] = df.index.map(Total_pronoun_counts)

    average = average_word_length(text)
    Average_Word_Length[i]=average 
    df['AVG WORD LENGTH'] = df.index.map(Average_Word_Length)
    print(url_id[i] +' Done !')
    res[f'{url_id[i]}'] = sia.polarity_scores(clean_text)

df.to_excel("Output Data Structure.xlsx")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


blackassign0001 Is Genarated in same folder !
blackassign0002 Is Genarated in same folder !
blackassign0003 Is Genarated in same folder !
blackassign0004 Is Genarated in same folder !
blackassign0005 Is Genarated in same folder !
blackassign0006 Is Genarated in same folder !
blackassign0007 Is Genarated in same folder !
blackassign0008 Is Genarated in same folder !
blackassign0009 Is Genarated in same folder !
blackassign0010 Is Genarated in same folder !
blackassign0011 Is Genarated in same folder !
blackassign0012 Is Genarated in same folder !
blackassign0013 Is Genarated in same folder !
blackassign0014 Is Genarated in same folder !
blackassign0015 Is Genarated in same folder !
blackassign0016 Is Genarated in same folder !
blackassign0017 Is Genarated in same folder !
blackassign0018 Is Genarated in same folder !
blackassign0019 Is Genarated in same folder !
blackassign0020 Is Genarated in same folder !
blackassign0021 Is Genarated in same folder !
blackassign0022 Is Genarated in sa

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


blackassign0001 Done !


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

blackassign0098 Done !


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


blackassign0099 Done !


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


blackassign0100 Done !
